In [7]:
import numpy             as np
import pandas            as pd
import matplotlib
import os
from   os      import getcwd
from   os      import listdir
from   os.path import isfile, join, isdir

import tensorflow as tf

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.metrics import Accuracy
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [8]:
def get_paths(foldNames):
  
    paths = dict.fromkeys(foldNames)

    for idx,g in enumerate(foldNames):
        fileNames = [f for f in listdir(join(trainPath,g)) if isfile(join(trainPath,g, f))]
        for i,f in enumerate(fileNames):
            fileNames[i] = join(trainPath,g,f)     
        paths[g] = fileNames
        
    return paths


# Setup

In [9]:
trainPath      = '../data/raw/train'
testPath       = '../data/raw/test_stg1'
rawdataPath    = '../data/raw'

fish_classes   = [f for f in listdir(trainPath) if isdir(join(trainPath, f))]
groupData      = pd.DataFrame ({'group': fish_classes})
fish_paths     = get_paths(fish_classes)

ROWS            = 32 # int(720  / downsample_amnt)
COLS            = 32 # int(1280 / downsample_amnt)
CHANNELS        = 3
NUM_CATEGORIES  = len(fish_classes)

## `build_hdf5_image_dataset`

In [10]:
%%time

BUILD_HDF5_DATASET = False
IMAGE_SIZE         = 32
VALIDATION_SPLIT   = True
output_path        = join(rawdataPath, 'fish_dataset_{}x{}.h5'.format(IMAGE_SIZE, IMAGE_SIZE))
input_path         = join(rawdataPath, 'train')

if BUILD_HDF5_DATASET:
    # Build a HDF5 dataset (only required once)
    from tflearn.data_utils import build_hdf5_image_dataset


    build_hdf5_image_dataset(target_path        =input_path, 
                             image_shape        =(IMAGE_SIZE, IMAGE_SIZE), 
                             mode               ='folder', 
                             output_path        =output_path, 
                             categorical_labels =True, 
                             normalize          =True)

CPU times: user 9 µs, sys: 3 µs, total: 12 µs
Wall time: 12.9 µs


In [11]:
%%time
from   sklearn.model_selection    import train_test_split

# Load HDF5 dataset
import h5py

h5f         = h5py.File(output_path, 'r')
X_all       = h5f['X'][()]
Y_all       = h5f['Y'][()]

# Split into 
if VALIDATION_SPLIT:
    X, X_valid, Y, Y_valid = train_test_split(X_all, Y_all, 
                                                          test_size    =0.2, 
                                                          random_state =23, 
                                                          stratify     =Y_all)


CPU times: user 123 ms, sys: 51.7 ms, total: 174 ms
Wall time: 169 ms


## `test model`

In [7]:
def dnn_test1():
    #needed to run this tensorflow operation in order to build the network and subsequently 
    #create the model, multiple times. Rebuilding without resetting the tf.Graph object produces
    #errors. Could also get around this issue by restarting kernel, but that's annoying.
    
    #python with statement: it will use this session for the code following the with statement 
    # and then automatically close the session
    with tf.Graph().as_default():
#         n_nodes_hl1 = 500
#         n_nodes_hl2 = 500
#         n_nodes_hl3 = 500
#         n_classes = 10
#         batch_size = 100
    
        # normalisation of images
        img_prep = ImagePreprocessing()
        img_prep.add_featurewise_zero_center()
        img_prep.add_featurewise_stdnorm()

        # Create extra synthetic training data by flipping & rotating images
        img_aug = ImageAugmentation()
        img_aug.add_random_flip_leftright()
        img_aug.add_random_rotation(max_angle=25.)

        
        #specific a specific device or gpu
#           with tf.device("/gpu:1"): 
        
        #input layer with shape of data specified. In this case, dimensions of our images, 
        #rows X cols X rgb array. The initial 'None' is for an unknown dimension reflecting the 
        #"number of samples that are processed in a batch"
        network = input_data(shape=[None, ROWS, COLS, 3],
                        data_preprocessing=img_prep,
                        data_augmentation=img_aug)
        
        # 1: Convolution layer with 32 filters, each 3x3x3
        conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1')

        # 2: Max pooling layer
        network = max_pool_2d(conv_1, 2)

        # 3: Convolution layer with 64 filters
        conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2')

        # 4: Convolution layer with 64 filters
        conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3')

        # 5: Max pooling layer
        network = max_pool_2d(conv_3, 2)

        # 6: Fully-connected 512 node layer
        network = fully_connected(network, 512, activation='relu')

        # 7: Dropout layer to combat overfitting
        network = dropout(network, 0.5)

        #output latyer
        network = fully_connected(network, 8, activation='softmax')
        
        # Configure how the network will be trained
        acc = Accuracy(name="Accuracy")
        
        network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)
        return tflearn.DNN(network)


In [9]:
# Define 

model_tf = dnn_test1()

# Start training (apply gradient descent algorithm). Will want to specify multiple epochs 
# typically unless just testing
model_tf.fit(X, Y, n_epoch=1,validation_set=(X_valid,Y_valid),
          show_metric=True, batch_size=15)

Training Step: 201  | total loss: 1.17208 | time: 3.728s
| Adam | epoch: 001 | loss: 1.17208 - Accuracy: 0.6038 -- iter: 3015/3021
Training Step: 202  | total loss: 1.14748 | time: 4.792s
| Adam | epoch: 001 | loss: 1.14748 - Accuracy: 0.6168 | val_loss: 1.13318 - val_acc: 0.6071 -- iter: 3021/3021
--


## `Train a model on the CIFAR-10 dataset and save it` 

In [3]:
# Data loading and preprocessing
from tflearn.datasets import cifar10

(CX, CY), (CX_test, CY_test) = cifar10.load_data()
CX, CY = shuffle(CX, CY)
CY = to_categorical(CY, 10)
CY_test = to_categorical(CY_test, 10)

In [4]:
with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model = tflearn.DNN(network, tensorboard_verbose=0,checkpoint_path='cifar10_cnn.tfl.ckpt')

    # Train using classifier
    model.fit(CX, CY, n_epoch=1, shuffle=True, validation_set=(CX_test, CY_test),
              show_metric=True, batch_size=100, run_id='cifar10_cnn')

    model.save("cifar10_cnn.tfl")

Training Step: 499  | total loss: 1.27679 | time: 19.662s
| Adam | epoch: 001 | loss: 1.27679 - acc: 0.5541 -- iter: 49900/50000
Training Step: 500  | total loss: 1.26675 | time: 21.106s
| Adam | epoch: 001 | loss: 1.26675 - acc: 0.5507 | val_loss: 1.15872 - val_acc: 0.5861 -- iter: 50000/50000
--
INFO:tensorflow:/Users/stokesjd/repos/fish/sashimdig/notebooks/cifar10_cnn.tfl.ckpt-500 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/Users/stokesjd/repos/fish/sashimdig/notebooks/cifar10_cnn.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model_ref = tflearn.DNN(network, tensorboard_verbose=0)
    model_ref.load('cifar10_cnn.tfl')
    # Train using classifier
    model_ref.fit(CX, CY, n_epoch=1, shuffle=True, validation_set=(CX_test, CY_test),
              show_metric=True, batch_size=100, run_id='cifar10_cnn_refine')


Training Step: 999  | total loss: 1.05309 | time: 19.045s
| Adam | epoch: 001 | loss: 1.05309 - acc: 0.6317 -- iter: 49900/50000
Training Step: 1000  | total loss: 1.04628 | time: 20.494s
| Adam | epoch: 001 | loss: 1.04628 - acc: 0.6335 | val_loss: 0.93027 - val_acc: 0.6738 -- iter: 50000/50000
--


In [16]:
with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, NUM_CATEGORIES, activation='softmax',restore='False')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model_ref = tflearn.DNN(network, tensorboard_verbose=0)
    model_ref.load('cifar10_cnn.tfl')
    # Train using classifier
    model_ref.fit(X, Y, n_epoch=1, shuffle=True, validation_set=(X_valid, Y_valid),
              show_metric=True, batch_size=100, run_id='fish_cnn_refine')


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [8] rhs shape= [10]
	 [[Node: save_1/Assign_39 = Assign[T=DT_FLOAT, _class=["loc:@FullyConnected_1/b"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](FullyConnected_1/b/Adam_1, save_1/RestoreV2_39/_9)]]

Caused by op u'save_1/Assign_39', defined at:
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/stokesjd/anaconda/envs/tensorflow2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/stokesjd/anaconda/e

In [ ]:
# model = tflearn.DNN(regression, checkpoint_path='model_finetuning',
#                     max_checkpoints=3, tensorboard_verbose=0)